In [1]:
import requests
from bs4 import BeautifulSoup


# Этот блокнот собирает данные по клиническим исследованиям в РФ с веб-страницы http://nik2.itsvc.ru 




In [7]:
# Cписки, это будущие колонки таблицы
date = []
fase = []
link = []
field = []
tipe = []
non = []
non2n = [] 
# Cкрипт собирающий данные с http://nik2.itsvc.ru 
for i in range(9):
    url = 'http://nik2.itsvc.ru/ki?title=&field_oblprimenenia_tid=All&page=0%d' % (i)
    r = requests.get(url)
    page = BeautifulSoup(r.text, 'html.parser')
    table = page.tbody #Таблицы обрамлены тегом 'tbody', помещаются в переменную

    # Проход по строкам
    for row in table.find_all('tr')[:]:
        
    # Выделение нужных ячеек по тегам и индексам, помещение их в соотвествующую колонку (список) 
        col = row.find_all('td')
        col_l= row.find_all('a')
        col_d= row.find_all('span')
        

        
        column_1 = col_l[0].string.strip()
        
        link.append(column_1)

        
        column_2 = col[1].string.strip()
        
        fase.append(column_2)

        
        column_3 = col_d[0].string.strip()
        
        date.append(column_3)

        
        column_4 = col[3].string.strip()
        
        field.append(column_4)

        
        column_5 = col[4].string.strip()
        
        tipe.append(column_5)

        
        column_7 = col_l[0] ['href']#.string.strip()
        
        non.append(column_7)

        
        column_8 = col[6]
        
        non2n.append(column_8)

    

In [9]:
import pandas as pd
# Названия колонок и присваивание им значений (полученных списков )
columns = {'date': date, 'fase': fase, 'link': link, 'field': field, 'tipe': tipe, 'non': non}#, 'non2n': non2n

# Табличка в датафрейм
df = pd.DataFrame(columns)

In [10]:
df.shape

(889, 6)

In [11]:
df.head()

,date,fase,field,link,non,tipe
0,08.09.2015,IIa,ММКИ,CNTO136-MDD-2001,/node/7816,Иммунология
1,01.09.2015,IIb,ММКИ,CQGE031B2204,/node/7784,Пульмонология
2,27.08.2015,III,ММКИ,MYL-1401A-3001,/node/7787,Дерматология
3,01.09.2015,IIIb,ММКИ,A4061008,/node/7788,Онкология
4,01.10.2015,III,ММКИ,РМ1331,/node/7771,Дерматология


In [12]:
#Сохраняем в файл
df.to_csv('clin_res.csv', index=True)

In [22]:
# В полученной таблице, в колонке 'non', внутренний линк с характеристиками исследования.
# Далее идет скрипт, который берет эти линки, ходит по ним и собирает данные в датафрейм.

date_s=[]
date_e=[]
nom_r=[]
nom_p=[]
nom_rki=[]
prt=[]
cntr=[]
org=[]
subj=[]
rschrs=[]
#Проход по линкам
for linkz in df.non:
    #Получение линка 
    url2='http://nik2.itsvc.ru/'+linkz
    #Получение данных по линку
    g= requests.get(url2)
    #Оформление (обрамление) данных
    page2 = BeautifulSoup(g.text, 'html.parser')
    #Выделение нужных ячеек по тегам и индексам, помещение их в соотвествующую колонку (список)
    dt=page2.find_all('span')
    column_10=dt[3].string.strip()
    date_s.append(column_10)
    column_11=dt[4].string.strip()
    date_e.append(column_11)
    
    dn = page2.find_all('div', class_="field-item even")  
    column_12 = dn[2].string.strip()
    nom_p.append(column_12)
    column_13 = dn[3].string.strip()
    nom_rki.append(column_13)
    column_14 = dn[4].string.strip()
    prt.append(column_14)
    
    #Забирает из таблицы (если она есть), кол-во исследователей, испытуемых, субъектов РФ
    drr = page2.find_all('li', class_="taxonomy-term-reference-0") 
    column_16=drr[1].string.strip()
    cntr.append(column_16)
    lst1=[]
    lst2=[]
    lst3=[]
    table3 = page2.tbody
    if table3!=None:
        for rowz in table3.find_all('tr')[:]:
            col2 = rowz.find_all('td')
            col3=rowz.find_all('a')

            column_17=col3[0].string.strip()
            lst1.append(column_17)
            column_18=col3[1].string.strip()
            lst2.append(column_18)
            column_19=col2[2].string.strip()
            lst3.append(column_19)
        org.append(lst1)
        subj.append(lst3)
        rschrs.append(lst2)
        nom_r.append(len(lst2))
    else:
        org.append(lst1)
        subj.append(lst3)
        rschrs.append(lst2)
        nom_r.append(len(lst2))

In [31]:
# Собирается второй датафрейм, более полный
columns2 = {'Начало иссл.': date_s, 'Конец иссл.': date_e, 'Кол-во исследователей': nom_r, 'Кол-во испытуемых': nom_p, 'nom_rki': nom_rki, 'Протокол': prt,'Организации': org, 'Субъекты РФ': subj,'Страна': cntr,  'Исследователи' : rschrs}
df2 = pd.DataFrame(columns2)
df2.shape

(889, 10)

In [32]:
df2.head()

,nom_rki,Исследователи,Кол-во испытуемых,Кол-во исследователей,Конец иссл.,Начало иссл.,Организации,Протокол,Страна,Субъекты РФ
0,491,"[Смулевич Анатолий Болеславович, Агарков Алекс...",180,12,01.04.2017,08.09.2015,"[ПКБ №1 им. Н.А. Алексеева, НИИ психического з...","№ CNTO136-MDD-2001 ""Многоцентровое, двойное сл...",Бельгия,"[г. Москва, Томская область, Смоленская област..."
1,471,"[Шапорова Наталия Леонидовна, Астафьева Наталь...",90,5,31.12.2017,01.09.2015,"[СПб ГУЗ«ГКДЦ №1», ГОУ ВПО Саратовский ГМУ им....","№ CQGE031B2204 ""«Многоцентровое, рандомизирова...",Швейцария,"[г. Санкт - Петербург, Саратовская область, Ря..."
2,468,"[Халилов Булат Василович, Самцов Алексей Викто...",71,8,15.06.2017,27.08.2015,[ГБОУ ВПО Казанский ГМУ Минздравсоцразвития Ро...,"№ MYL-1401A-3001 ""Многоцентровое, двойное слеп...",Швейцария,"[Республика Татарстан (Татарстан), г. Санкт - ..."
3,467,"[Демидов Лев Вадимович, Бычков Юрий Маркович, ...",13,3,21.06.2017,01.09.2015,"[ФГБНУ «РОНЦ им. Н.Н. Блохина», ФГУ «РНЦРР» Ми...",№ A4061008 «Продление доступа к препарату AG-0...,США,"[г. Москва, г. Москва, г. Санкт - Петербург]"
4,484,"[Кубанов Алексей Алексеевич, Рахматулина Марга...",150,7,01.12.2017,01.10.2015,"[ГБОУ ДПО РМАПО Минздравсоцразвития России, ФГ...","№ РМ1331 ""Многоцентровое, рандомизированное, д...",Швейцария,"[г. Москва, г. Москва, г. Санкт - Петербург, г..."


In [33]:
#Сохраняем в файл
df2.to_csv('clin_res2.csv', index=True)